## Reading TSPLIB data

This notebook is for experiment with :
    1. Methods to extracting the data from TSPLIB dataset into a usable format
    2. Simulated annealing for TSP with QUBO and without.

#### Convert the tsp file into list

In [1]:
file1 = open('bays29.tsp')     #Test Dataset from Symmetric TSP
lines_list = file1.read().splitlines()

In [2]:
distance_list = lines_list[8:37]    #29 lines for distance values of each 29x29 node pairs
nodes_list = lines_list[38:-1]      #29 lines for the nodes coordinate values
print(distance_list)
print(nodes_list)

['   0 107 241 190 124  80 316  76 152 157 283 133 113 297 228 129 348 276 188 150  65 341 184  67 221 169 108  45 167', ' 107   0 148 137  88 127 336 183 134  95 254 180 101 234 175 176 265 199 182  67  42 278 271 146 251 105 191 139  79', ' 241 148   0 374 171 259 509 317 217 232 491 312 280 391 412 349 422 356 355 204 182 435 417 292 424 116 337 273  77', ' 190 137 374   0 202 234 222 192 248  42 117 287  79 107  38 121 152  86  68  70 137 151 239 135 137 242 165 228 205', ' 124  88 171 202   0  61 392 202  46 160 319 112 163 322 240 232 314 287 238 155  65 366 300 175 307  57 220 121  97', '  80 127 259 234  61   0 386 141  72 167 351  55 157 331 272 226 362 296 232 164  85 375 249 147 301 118 188  60 185', ' 316 336 509 222 392 386   0 233 438 254 202 439 235 254 210 187 313 266 154 282 321 298 168 249  95 437 190 314 435', '  76 183 317 192 202 141 233   0 213 188 272 193 131 302 233  98 344 289 177 216 141 346 108  57 190 245  43  81 243', ' 152 134 217 248  46  72 438 213   0 2

#### Convert the Lists into 2D Matrix

In [3]:
# The list containg the lines are split and null values are removed
distance_matrix = [0 for i in range (29)]
nodes_matrix = [0 for i in range(29)]
for i in range(29):
    d_row = distance_list[i].split(' ')
    d_row = list(filter(None,d_row))
    n_row = nodes_list[i].split(' ')
    n_row = list(filter(None,n_row))
    distance_matrix[i] = d_row
    nodes_matrix[i] = n_row
print(distance_matrix)
print(nodes_matrix)
    

[['0', '107', '241', '190', '124', '80', '316', '76', '152', '157', '283', '133', '113', '297', '228', '129', '348', '276', '188', '150', '65', '341', '184', '67', '221', '169', '108', '45', '167'], ['107', '0', '148', '137', '88', '127', '336', '183', '134', '95', '254', '180', '101', '234', '175', '176', '265', '199', '182', '67', '42', '278', '271', '146', '251', '105', '191', '139', '79'], ['241', '148', '0', '374', '171', '259', '509', '317', '217', '232', '491', '312', '280', '391', '412', '349', '422', '356', '355', '204', '182', '435', '417', '292', '424', '116', '337', '273', '77'], ['190', '137', '374', '0', '202', '234', '222', '192', '248', '42', '117', '287', '79', '107', '38', '121', '152', '86', '68', '70', '137', '151', '239', '135', '137', '242', '165', '228', '205'], ['124', '88', '171', '202', '0', '61', '392', '202', '46', '160', '319', '112', '163', '322', '240', '232', '314', '287', '238', '155', '65', '366', '300', '175', '307', '57', '220', '121', '97'], ['80', 

In [4]:
# reshaping the matrices
import numpy as np
#Convert the lists to numpy arrays
distance_matrix = np.array(distance_matrix)
nodes_matrix = np.array(nodes_matrix)
distance_matrix = distance_matrix.reshape(29,29)        # Re-shape
nodes_matrix = nodes_matrix.reshape(29,3)               # Re-shape
distance_matrix = distance_matrix.astype(np.float)      # The elements of array were of string type
nodes_matrix = nodes_matrix.astype(np.float)            # So we convert them to float

C:\Users\HP\AppData\Local\Temp\ipykernel_14124\2039403970.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  distance_matrix = distance_matrix.astype(np.float)      # The elements of array were of string type
C:\Users\HP\AppData\Local\Temp\ipykernel_14124\2039403970.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  nodes_matrix = nodes

In [5]:
# check the matrices
print(distance_matrix)
print(nodes_matrix)

[[  0. 107. 241. 190. 124.  80. 316.  76. 152. 157. 283. 133. 113. 297.
  228. 129. 348. 276. 188. 150.  65. 341. 184.  67. 221. 169. 108.  45.
  167.]
 [107.   0. 148. 137.  88. 127. 336. 183. 134.  95. 254. 180. 101. 234.
  175. 176. 265. 199. 182.  67.  42. 278. 271. 146. 251. 105. 191. 139.
   79.]
 [241. 148.   0. 374. 171. 259. 509. 317. 217. 232. 491. 312. 280. 391.
  412. 349. 422. 356. 355. 204. 182. 435. 417. 292. 424. 116. 337. 273.
   77.]
 [190. 137. 374.   0. 202. 234. 222. 192. 248.  42. 117. 287.  79. 107.
   38. 121. 152.  86.  68.  70. 137. 151. 239. 135. 137. 242. 165. 228.
  205.]
 [124.  88. 171. 202.   0.  61. 392. 202.  46. 160. 319. 112. 163. 322.
  240. 232. 314. 287. 238. 155.  65. 366. 300. 175. 307.  57. 220. 121.
   97.]
 [ 80. 127. 259. 234.  61.   0. 386. 141.  72. 167. 351.  55. 157. 331.
  272. 226. 362. 296. 232. 164.  85. 375. 249. 147. 301. 118. 188.  60.
  185.]
 [316. 336. 509. 222. 392. 386.   0. 233. 438. 254. 202. 439. 235. 254.
  210. 187. 313.

## TSP to QUBO
#### Custom function for QUBO Formulation compatible with dwave SimulatedAnnealingSampler
#### parts of dimod are getting deprecated so let us use dwave-neal instead

In [35]:
import dimod

def tsp_qubo(distance_matrix):
    num_cities = len(distance_matrix)
    qubo = {}

    # Variable definition
    for i in range(num_cities):
        for j in range(num_cities):
            if i == j:
                qubo[(i, j)] = distance_matrix[i][j]
            else:
                qubo[(i, j)] = 0.0

    # Constraint: Each city should be visited exactly once
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                for k in range(num_cities):
                    if k != i and k != j:
                        qubo[(i, j)] += 2 * distance_matrix[i][k]

    return qubo


# FOR TESTING PURPOSES==============================================================================

# The distance matrix of the first 10 nodes
test_matrix = distance_matrix[:10 , :10]     

# Formulate TSP as a QUBO problem
qubo_matrix = tsp_qubo(test_matrix)

# Create a BQM object for the QUBO problem
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo_matrix)

# Print the QUBO matrix
elist = [0 for i in range(29)]
print(qubo_matrix)
print("Binary Quadratic Model:")
for (i, j), value in bqm.quadratic.items():
    print(f"{i} -> {j}: {value}")
for i, bias in bqm.linear.items():
    print(f"{i}: {bias}")


{(0, 0): 0.0, (0, 1): 2672.0, (0, 2): 2404.0, (0, 3): 2506.0, (0, 4): 2638.0, (0, 5): 2726.0, (0, 6): 2254.0, (0, 7): 2734.0, (0, 8): 2582.0, (0, 9): 2572.0, (1, 0): 2496.0, (1, 1): 0.0, (1, 2): 2414.0, (1, 3): 2436.0, (1, 4): 2534.0, (1, 5): 2456.0, (1, 6): 2038.0, (1, 7): 2344.0, (1, 8): 2442.0, (1, 9): 2520.0, (2, 0): 4454.0, (2, 1): 4640.0, (2, 2): 0.0, (2, 3): 4188.0, (2, 4): 4594.0, (2, 5): 4418.0, (2, 6): 3918.0, (2, 7): 4302.0, (2, 8): 4502.0, (2, 9): 4472.0, (3, 0): 3302.0, (3, 1): 3408.0, (3, 2): 2934.0, (3, 3): 0.0, (3, 4): 3278.0, (3, 5): 3214.0, (3, 6): 3238.0, (3, 7): 3298.0, (3, 8): 3186.0, (3, 9): 3598.0, (4, 0): 2644.0, (4, 1): 2716.0, (4, 2): 2550.0, (4, 3): 2488.0, (4, 4): 0.0, (4, 5): 2770.0, (4, 6): 2108.0, (4, 7): 2488.0, (4, 8): 2800.0, (4, 9): 2572.0, (5, 0): 2894.0, (5, 1): 2800.0, (5, 2): 2536.0, (5, 3): 2586.0, (5, 4): 2932.0, (5, 5): 0.0, (5, 6): 2282.0, (5, 7): 2772.0, (5, 8): 2910.0, (5, 9): 2720.0, (6, 0): 5540.0, (6, 1): 5500.0, (6, 2): 5154.0, (6, 3): 5

## Solve QUBO using SA

In [36]:
from dwave.samplers import SimulatedAnnealingSampler

sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, beta_range=[.1, 4.2], num_reads=100, beta_schedule_type='linear')

In [37]:
print(sampleset)

    0  1  2  3  4  5  6  7  8  9 energy num_oc.
0   0  0  0  0  0  0  0  0  1  0    0.0       1
1   0  0  0  0  0  0  1  0  0  0    0.0       1
2   0  0  0  1  0  0  0  0  0  0    0.0       1
3   0  0  0  0  0  0  0  0  1  0    0.0       1
4   0  0  0  0  0  0  0  1  0  0    0.0       1
5   0  0  0  0  0  0  0  0  1  0    0.0       1
6   0  0  0  0  0  0  1  0  0  0    0.0       1
7   0  0  0  0  0  0  0  1  0  0    0.0       1
8   0  0  0  0  0  0  0  0  1  0    0.0       1
9   0  0  0  0  0  0  0  0  1  0    0.0       1
10  0  0  0  0  0  0  0  0  1  0    0.0       1
11  0  0  0  0  1  0  0  0  0  0    0.0       1
12  0  0  0  0  1  0  0  0  0  0    0.0       1
13  0  0  0  0  0  0  0  1  0  0    0.0       1
14  0  0  0  0  0  0  0  1  0  0    0.0       1
15  0  0  0  0  0  0  1  0  0  0    0.0       1
16  0  0  0  0  0  1  0  0  0  0    0.0       1
17  0  0  0  0  0  0  0  0  1  0    0.0       1
18  0  0  0  0  0  0  0  1  0  0    0.0       1
19  0  0  0  0  0  0  0  0  1  0    0.0 

## QUBO formulation using PyQUBO
### Docs: https://pyqubo.readthedocs.io/en/latest/index.html
<img src="https://raw.githubusercontent.com/recruit-communications/pyqubo/master/images/benchmark.svg" /> 

#### PyQUBO has been benchmarked on TSP problems and has a significant execution time advantage and is highly optimised.

#### Reference for code : https://github.com/recruit-communications/pyqubo/blob/master/notebooks/TSP.ipynb

In [ ]:
%matplotlib inline
from pyqubo import Array, Placeholder, Constraint
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
def plot_city(cities, sol=None):
    n_city = len(cities)
    cities_dict = dict(cities)
    G = nx.Graph()
    for city in cities_dict:
        G.add_node(city)
        
    # draw path
    if sol:
        city_order = []
        for i in range(n_city):
            for j in range(n_city):
                if sol.array('c', (i, j)) == 1:
                    city_order.append(j)
        for i in range(n_city):
            city_index1 = city_order[i]
            city_index2 = city_order[(i+1) % n_city]
            G.add_edge(cities[city_index1][0], cities[city_index2][0])

    plt.figure(figsize=(3,3))
    pos = nx.spring_layout(G)
    nx.draw_networkx(G, cities_dict)
    plt.axis("off")
    plt.show()

def dist(i, j, cities):
    pos_i = cities[i][1]
    pos_j = cities[j][1]
    return np.sqrt((pos_i[0] - pos_j[0])**2 + (pos_i[1] - pos_j[1])**2)

### NOTE TO SELF: Insert in the following code instance from the dataset

In [ ]:
# City names and coordinates list[("name", (x, y))]
cities = [
    ("a", (0, 0)),
    ("b", (1, 3)),
    ("c", (3, 2)),
    ("d", (2, 1)),
    ("e", (0, 1))
]
plot_city(cities)

Prepare binary vector with bit (𝑖,𝑗) representing to visit 𝑗 city at time 𝑖

In [ ]:
n_city = len(cities)
x = Array.create('c', (n_city, n_city), 'BINARY')

In [ ]:
# Constraint not to visit more than two cities at the same time.
time_const = 0.0
for i in range(n_city):
    # If you wrap the hamiltonian by Const(...), this part is recognized as constraint
    time_const += Constraint((sum(x[i, j] for j in range(n_city)) - 1)**2, label="time{}".format(i))

# Constraint not to visit the same city more than twice.
city_const = 0.0
for j in range(n_city):
    city_const += Constraint((sum(x[i, j] for i in range(n_city)) - 1)**2, label="city{}".format(j))

In [ ]:
# distance of route
distance = 0.0
for i in range(n_city):
    for j in range(n_city):
        for k in range(n_city):
            d_ij = dist(i, j, cities)
            distance += d_ij * x[k, i] * x[(k+1)%n_city, j]

In [ ]:
# Construct hamiltonian
A = Placeholder("A")
H = distance + A * (time_const + city_const)

In [ ]:
# Compile model
model = H.compile()

In [ ]:
# Generate QUBO
feed_dict = {'A': 4.0}
bqm = model.to_bqm(feed_dict=feed_dict)

In [ ]:
import neal
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=100, num_sweeps=100)

# Decode solution
decoded_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
best_sample = min(decoded_samples, key=lambda x: x.energy)
num_broken = len(best_sample.constraints(only_broken=True))
print("number of broken constarint = {}".format(num_broken))

In [ ]:
if num_broken == 0:
    plot_city(cities, best_sample)

## Solve TSP instance with SA (without QUBO formulation)